In [1]:
import os
import json
from tqdm import tqdm
from dotenv import load_dotenv

from openai import OpenAI
from elasticsearch import Elasticsearch

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [3]:
# load the running qa documents
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
 'section': 'Training',
 'question': 'Was ist Intervalltraining?',
 'course': 'running-assistant-rag'}

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
es_client.info()

ObjectApiResponse({'name': '5e6af68a49e7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '-zlyx-UFQ6SmB_FcuOnFtw', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "running-questions"

In [ ]:
# uncomment to delete index
# if es_client.indices.exists(index=index_name):
    # es_client.indices.delete(index=index_name)

# only need to run when you run the notebook for the first time to create your index. Once the index is created, you don't need to set the index again
es_client.indices.create(index=index_name, body=index_settings, request_timeout=60)

In [33]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc, request_timeout=60)

  0%|                                                | 0/98 [00:00<?, ?it/s]/tmp/ipykernel_11373/2316373423.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.index(index=index_name, document=doc, request_timeout=60)
100%|███████████████████████████████████████| 98/98 [00:01<00:00, 73.99it/s]


In [10]:
query = 'Was ist Intervalltraining?'

In [11]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "running-assistant-rag"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [12]:
# search in running qa documents with elasticsearch
elastic_search('Wie fange ich mit dem Laufen an?')

[{'text': 'Barfußlaufen fördert die Fußmuskulatur, sollte aber langsam aufgebaut werden, um Verletzungen zu vermeiden.',
  'section': 'Training',
  'question': 'Wie beginnt man mit Barfußlaufen sicher?',
  'course': 'running-assistant-rag'},
 {'text': 'Das Tragen von reflektierender Kleidung erhöht die Sichtbarkeit und Sicherheit beim Laufen in der Dunkelheit.',
  'section': 'Ausrüstung',
  'question': 'Wie kann ich beim Laufen im Dunkeln sicherer sein?',
  'course': 'running-assistant-rag'},
 {'text': 'Der optimale Laufschuh sollte gut passen, eine ausreichende Dämpfung bieten und zu deinem Laufstil passen, um Verletzungen vorzubeugen.',
  'section': 'Ausrüstung',
  'question': 'Wie finde ich den richtigen Laufschuh?',
  'course': 'running-assistant-rag'},
 {'text': 'Das richtige Tempo finden ist wichtig, um das Rennen effektiv durchzuhalten und das Risiko eines Leistungseinbruchs zu minimieren.',
  'section': 'Wettkämpfe',
  'question': 'Wie finde ich das richtige Renntempo?',
  'cou

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
You are a personal trainer specialized in running.

Answer the QUESTION at the end based on the full CONTEXT below, which contains multiple relevant FAQ entries.
Use **all available context entries** to form your answer. If multiple answers apply, summarize or combine them clearly.
Don't mention the Entry number like [Entry 5] in your answer.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context_entries = []
    
    for i, doc in enumerate(search_results, start=1):
        entry = (
            f"[Entry {i}]\n"
            f"Section: {doc['section']}\n"
            f"Question: {doc['question']}\n"
            f"Answer: {doc['text']}"
        )
        context_entries.append(entry)

    context = "\n\n".join(context_entries)
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
def llm(prompt):
    # uncomment to see full prompt
    # print(prompt)
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}],
        temperature=1.5,
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        max_tokens=1000,
    )
    
    return response.choices[0].message.content

In [15]:
def rag_pipeline(query):
    context_docs = elastic_search(query)
    prompt = build_prompt(query, context_docs)
    answer = llm(prompt)
    return answer

In [16]:
query = 'Wie fange ich mit dem Laufen an?'

In [26]:
# answer using qa documents for rag
rag_pipeline(query)

'Um mit dem Laufen anzufangen, ist es wichtig, langsam zu starten und den Körper schrittweise an die neue Aktivität zu gewöhnen, um Verletzungen zu vermeiden. Beginnen Sie mit kurzen Strecken und steigern Sie allmählich die Distanz und Intensität Ihres Lauftrainings. Achten Sie darauf, sich vor und nach dem Laufen angemessen aufzuwärmen und zu dehnen, wobei das Dehnen nach dem Lauf helfen kann, die Beweglichkeit zu fördern und die Regeneration zu unterstützen. Eine wichtige Komponente ist auch das Tragen geeigneter Laufschuhe, die gut passen, ausreichende Dämpfung bieten und Ihrem individuellen Laufstil entsprechen. Dies trägt ebenfalls dazu bei, Verletzungen zu verhindern.'

In [23]:
# answer from gpt-4o without using rag
llm(query)

'Mit dem Laufen anzufangen ist eine großartige Entscheidung für Ihre Gesundheit und Ihr Wohlbefinden. Hier sind einige Tipps, um Ihnen den Einstieg zu erleichtern:\n\n1. **Ziele setzen:** Definieren Sie klare, realistische Ziele. Möchten Sie fitter werden, abnehmen oder an einem Wettlauf teilnehmen? Dies hilft Ihnen, motiviert zu bleiben.\n\n2. **Richtige Ausrüstung:** Investieren Sie in ein Paar gute Laufschuhe, die zu Ihrem Fußtyp und Laufstil passen. Fachgeschäfte bieten oft eine Beratung und Analyse an.\n\n3. **Langsam anfangen:** Beginnen Sie mit einem Wechsel aus Laufen und Gehen. Zum Beispiel: 1 Minute laufen, 2 Minuten gehen, und das für 20-30 Minuten. Steigern Sie langsam die Laufanteile.\n\n4. **Aufwärmen und Abkühlen:** Starten Sie jede Laufeinheit mit einem Aufwärmen, zum Beispiel leichtes Gehen und dynamische Dehnübungen. Vergessen Sie nicht, am Ende zu dehnen, um die Regeneration zu fördern.\n\n5. **Regelmäßigkeit:** Versuchen Sie, regelmäßig zu laufen, am besten 3-mal pr